# Chapter 6: Classes and Objects

Object-Oriented Programming (OOP) is a paradigm that organizes code around **objects** rather than functions and logic. An object encapsulates data (attributes) and behavior (methods) into a single unit. Python's approach to OOP is elegant and pragmatic, offering powerful features like multiple inheritance, operator overloading, and introspection while maintaining readability.

This chapter will guide you through creating classes, managing inheritance hierarchies, controlling access to data, and implementing Python's protocol-based object model. We will emphasize modern patterns including type hints, dataclasses, and abstract base classes that represent current industry standards.

## 6.1 Classes and Instances

A **class** is a blueprint for creating objects; an **instance** is a specific object created from that blueprint.

### Defining a Class

```python
from datetime import datetime
from typing import Optional

class User:
    """
    Represents a user in the system.
    
    Attributes:
        username: Unique identifier for the user
        email: User's email address
        created_at: Timestamp when the user was created
    """
    
    # Class attribute - shared across all instances
    domain: str = "example.com"
    
    def __init__(self, username: str, email: str) -> None:
        """
        Initialize a new User instance.
        
        Args:
            username: The user's unique username
            email: The user's email address
        """
        # Instance attributes - unique to each instance
        self.username: str = username
        self.email: str = email
        self.created_at: datetime = datetime.now()
        self._is_active: bool = True  # Protected attribute convention
    
    def greet(self) -> str:
        """Return a greeting message."""
        return f"Hello, {self.username}!"
    
    def deactivate(self) -> None:
        """Deactivate the user account."""
        self._is_active = False
```

**Key Concepts:**
*   **`class` keyword**: Defines a new class.
*   **`__init__`**: The initializer (constructor). It runs when an instance is created to set up initial state. Note that `__new__` actually creates the instance, but `__init__` initializes it—99% of the time you only need `__init__`.
*   **`self`**: The first parameter of instance methods refers to the instance itself. It is passed implicitly when you call the method.
*   **Class attributes**: Defined directly in the class body, shared by all instances.
*   **Instance attributes**: Created via `self.attribute` assignment, unique to each instance.

### Creating Instances

```python
# Creating instances (objects)
alice: User = User("alice", "alice@example.com")
bob: User = User("bob", "bob@example.com")

# Accessing attributes
print(alice.username)  # "alice"
print(alice.created_at)  # Current timestamp

# Calling methods
message: str = alice.greet()  # "Hello, alice!"

# Modifying attributes
alice.email = "alice.smith@example.com"
```

**Type Safety with Classes:**
When using type checkers (mypy, pyright), always annotate the `__init__` parameters and return type (`-> None`).

## 6.2 Method Types

Python classes support three types of methods, distinguished by their first parameter and decorators.

### Instance Methods
The default type. They operate on instance data (access `self`).

```python
class BankAccount:
    def __init__(self, owner: str, balance: float = 0.0) -> None:
        self.owner: str = owner
        self.balance: float = balance
    
    def deposit(self, amount: float) -> None:
        """Instance method modifying instance state."""
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        self.balance += amount
    
    def get_balance(self) -> float:
        """Instance method accessing instance state."""
        return self.balance
```

### Class Methods (`@classmethod`)
Operate on the class itself, not instances. They receive the class as the first argument (conventionally named `cls`). Useful for alternative constructors.

```python
from datetime import date
from typing import Self  # Python 3.11+, or use TypeVar for older versions

class Employee:
    def __init__(self, name: str, start_date: date) -> None:
        self.name: str = name
        self.start_date: date = start_date
    
    @classmethod
    def from_string(cls, name: str, date_str: str) -> Self:
        """
        Alternative constructor parsing date from string.
        
        Usage: Employee.from_string("Alice", "2023-01-15")
        """
        year, month, day = map(int, date_str.split("-"))
        return cls(name, date(year, month, day))
    
    @classmethod
    def get_company_policy(cls) -> str:
        """Access class-level data."""
        return f"Policy for {cls.__name__}"

# Usage
emp: Employee = Employee.from_string("Alice", "2023-01-15")
```

### Static Methods (`@staticmethod`)
Neither receive `self` nor `cls`. They are utility functions that logically belong to the class namespace but don't access class or instance state.

```python
class TemperatureConverter:
    @staticmethod
    def celsius_to_fahrenheit(celsius: float) -> float:
        """Utility function - no instance/class data needed."""
        return (celsius * 9/5) + 32
    
    @staticmethod
    def fahrenheit_to_celsius(fahrenheit: float) -> float:
        return (fahrenheit - 32) * 5/9

# Usage (no instance creation needed)
temp_f: float = TemperatureConverter.celsius_to_fahrenheit(100)
```

**Decision Guide:**
*   Need instance data? → Instance method (no decorator)
*   Need class data or alternative constructor? → `@classmethod`
*   Neither? → `@staticmethod` (or consider if it belongs in the class at all)

## 6.3 Inheritance and Composition

**Inheritance** ("is-a" relationship) allows a class to inherit attributes and methods from a parent class.

```python
class Animal:
    """Base class."""
    def __init__(self, name: str, species: str) -> None:
        self.name: str = name
        self.species: str = species
    
    def make_sound(self) -> str:
        return "Some generic sound"
    
    def __str__(self) -> str:
        return f"{self.name} the {self.species}"

class Dog(Animal):
    """Derived class inherits from Animal."""
    def __init__(self, name: str, breed: str) -> None:
        # Call parent initializer
        super().__init__(name, species="Canis familiaris")
        self.breed: str = breed
    
    # Method overriding
    def make_sound(self) -> str:
        return "Woof!"
    
    def fetch(self, item: str) -> str:
        """New method specific to Dog."""
        return f"{self.name} fetched the {item}"

# Usage
buddy: Dog = Dog("Buddy", "Golden Retriever")
print(buddy.make_sound())  # "Woof!" (overridden method)
print(buddy.species)       # "Canis familiaris" (inherited attribute)
```

**The `super()` Function:**
`super()` returns a proxy object to the parent class, allowing you to call parent methods without explicitly naming the parent class. This is crucial for:
1.  Calling the parent's `__init__`
2.  Extending rather than replacing parent methods
3.  Cooperative multiple inheritance (see below)

### Composition Over Inheritance
While inheritance is powerful, **composition** ("has-a" relationship) is often more flexible and less coupled.

```python
# Inheritance approach (can become rigid)
class FlyingBird(Bird):
    def fly(self) -> None: ...

class Penguin(Bird):
    # Problem: Penguins are birds but can't fly
    def fly(self) -> None:
        raise NotImplementedError("Penguins can't fly")

# Composition approach (preferred for flexibility)
class FlyBehavior:
    def fly(self) -> None:
        raise NotImplementedError

class CanFly(FlyBehavior):
    def fly(self) -> None:
        print("Flying high!")

class CannotFly(FlyBehavior):
    def fly(self) -> None:
        print("I can't fly")

class Bird:
    def __init__(self, name: str, fly_behavior: FlyBehavior) -> None:
        self.name: str = name
        self.fly_behavior: FlyBehavior = fly_behavior
    
    def perform_fly(self) -> None:
        self.fly_behavior.fly()

# Usage
eagle: Bird = Bird("Eagle", CanFly())
penguin: Bird = Bird("Penguin", CannotFly())
```

**Industry Guideline:** Favor composition for code reuse. Use inheritance only when there is a true taxonomic "is-a" relationship and the Liskov Substitution Principle applies (a child class should be usable anywhere the parent is expected).

## 6.4 Multiple Inheritance and Method Resolution Order (MRO)

Python supports multiple inheritance (inheriting from multiple parents). This is powerful but requires understanding the **Method Resolution Order (MRO)**—the order in which Python looks for methods.

```python
class Flyer:
    def move(self) -> str:
        return "Flying through air"

class Swimmer:
    def move(self) -> str:
        return "Swimming in water"

class Duck(Flyer, Swimmer):
    def quack(self) -> str:
        return "Quack!"

# Which move() gets called?
donald: Duck = Duck()
print(donald.move())  # "Flying through air" (Flyer is first in MRO)
```

**The Diamond Problem:**
When classes inherit from a common ancestor through different paths, the MRO ensures each class is visited only once, from left to right, depth-first (with a twist for shared parents).

```
    A
   / \
  B   C
   \ /
    D
```

```python
class A:
    def method(self) -> str:
        return "A"

class B(A):
    def method(self) -> str:
        return "B"

class C(A):
    def method(self) -> str:
        return "C"

class D(B, C):
    pass

# Check MRO
print(D.__mro__)  # (<class 'D'>, <class 'B'>, <class 'C'>, <class 'A'>, <class 'object'>)
print(D().method())  # "B" (B is before C in the class definition)
```

**Using `super()` in Multiple Inheritance:**
`super()` doesn't just call the parent—it calls the **next class in the MRO**. This enables cooperative multiple inheritance where each class in the chain can contribute behavior.

```python
class Base:
    def __init__(self) -> None:
        print("Base init")
        super().__init__()  # Calls next in MRO (even though Base has no parent)

class Feature1(Base):
    def __init__(self) -> None:
        print("Feature1 init")
        super().__init__()

class Feature2(Base):
    def __init__(self) -> None:
        print("Feature2 init")
        super().__init__()

class Combined(Feature1, Feature2):
    def __init__(self) -> None:
        print("Combined init")
        super().__init__()  # Calls Feature1, which calls Feature2, which calls Base

# Output:
# Combined init
# Feature1 init
# Feature2 init
# Base init
```

**Best Practice:** Avoid deep multiple inheritance hierarchies. If you must use it, always use `super()` to ensure the MRO chain is respected, and check your class hierarchy with `ClassName.__mro__`.

## 6.5 Encapsulation and Access Control

Encapsulation is the bundling of data with the methods that operate on that data, and restricting direct access to some components. Python takes a "we're all consenting adults" approach—there are no true private members, only conventions.

### Naming Conventions

**Public:** No underscore prefix. `self.name`
*   Can be accessed from anywhere.

**Protected:** Single underscore prefix. `self._name`
*   Convention: "Internal use only, subclass access OK."
*   Not enforced by the language, but respected by tools and developers.

**Private:** Double underscore prefix. `self.__name`
*   Triggers **name mangling**: Python renames it to `_ClassName__name`.
*   Makes it harder (but not impossible) to access from outside the class.
*   Useful to prevent name clashes in subclasses.

```python
class BankAccount:
    def __init__(self, owner: str, balance: float) -> None:
        self.owner: str = owner           # Public
        self._balance: float = balance    # Protected (internal use)
        self.__pin: str = "1234"          # Private (name mangled)
    
    def get_balance(self) -> float:
        """Controlled access to balance."""
        return self._balance
    
    def _calculate_interest(self) -> float:
        """Protected method - subclass can override."""
        return self._balance * 0.02
    
    def __validate_pin(self, pin: str) -> bool:
        """Private method - subclass cannot easily override."""
        return pin == self.__pin

# Access demonstration
account: BankAccount = BankAccount("Alice", 1000)
print(account.owner)        # OK: "Alice"
print(account._balance)     # Possible but discouraged: 1000
# print(account.__pin)      # AttributeError: 'BankAccount' object has no attribute '__pin'
print(account._BankAccount__pin)  # Possible with mangled name: "1234"
```

**Industry Standard:** Use single underscore for internal API, double underscore only when necessary to avoid subclass conflicts. Never use double underscore to enforce "security"—it provides obfuscation, not protection.

## 6.6 Properties: Controlled Attribute Access

The `@property` decorator allows you to define methods that are accessed like attributes, enabling computed properties, validation, and read-only fields.

```python
class Temperature:
    def __init__(self, celsius: float = 0) -> None:
        self._celsius: float = celsius
    
    @property
    def celsius(self) -> float:
        """Get temperature in Celsius."""
        return self._celsius
    
    @celsius.setter
    def celsius(self, value: float) -> None:
        """Set temperature with validation."""
        if value < -273.15:
            raise ValueError("Temperature below absolute zero is not possible")
        self._celsius = value
    
    @property
    def fahrenheit(self) -> float:
        """Computed property - no setter means read-only."""
        return (self._celsius * 9/5) + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value: float) -> None:
        """Allow setting via Fahrenheit, converting to Celsius."""
        self.celsius = (value - 32) * 5/9

# Usage
temp: Temperature = Temperature(25)
print(temp.celsius)     # 25 (accessed like attribute, not method)
temp.celsius = 30       # Valid
print(temp.fahrenheit)  # 86.0 (computed)

temp.fahrenheit = 100   # Converts and sets
print(temp.celsius)     # 37.777...

# temp.celsius = -300   # Raises ValueError
```

**Benefits of Properties:**
1.  **Encapsulation**: Change internal representation without breaking external API.
2.  **Validation**: Enforce constraints on attribute assignment.
3.  **Computed Values**: Calculate on-the-fly without explicit method calls.
4.  **Caching**: Compute expensive values once, cache until invalidated.

## 6.7 Magic Methods (Dunder Methods)

**Magic methods** (double underscore methods like `__init__`, `__str__`) allow your classes to integrate with Python's syntax and built-in functions.

### Essential Magic Methods

```python
from typing import Any

class Book:
    def __init__(self, title: str, author: str, pages: int) -> None:
        self.title: str = title
        self.author: str = author
        self.pages: int = pages
    
    def __str__(self) -> str:
        """Informal string representation for users."""
        return f"{self.title} by {self.author}"
    
    def __repr__(self) -> str:
        """Official string representation for developers/debugging."""
        return f"Book(title='{self.title}', author='{self.author}', pages={self.pages})"
    
    def __eq__(self, other: Any) -> bool:
        """Equality comparison (==)."""
        if not isinstance(other, Book):
            return NotImplemented
        return (self.title == other.title and 
                self.author == other.author)
    
    def __lt__(self, other: Book) -> bool:
        """Less than comparison (<), enables sorting."""
        return self.pages < other.pages
    
    def __len__(self) -> int:
        """Support len() function."""
        return self.pages
    
    def __hash__(self) -> int:
        """
        Support use in sets and as dict keys.
        Only if immutable criteria met (or treated as immutable).
        """
        return hash((self.title, self.author))
    
    def __bool__(self) -> bool:
        """Truthiness check."""
        return self.pages > 0

# Usage
book: Book = Book("1984", "George Orwell", 328)
print(str(book))        # "1984 by George Orwell"
print(repr(book))       # "Book(title='1984', author='George Orwell', pages=328)"
print(len(book))        # 328
if book:                # Uses __bool__
    print("Book has content")
```

### Operator Overloading

Implement mathematical and bitwise operators:

```python
class Vector:
    def __init__(self, x: float, y: float) -> None:
        self.x: float = x
        self.y: float = y
    
    def __add__(self, other: Vector) -> Vector:
        """Enable v1 + v2 syntax."""
        return Vector(self.x + other.x, self.y + other.y)
    
    def __sub__(self, other: Vector) -> Vector:
        return Vector(self.x - other.x, self.y - other.y)
    
    def __mul__(self, scalar: float) -> Vector:
        """Scalar multiplication: v * 3."""
        return Vector(self.x * scalar, self.y * scalar)
    
    def __rmul__(self, scalar: float) -> Vector:
        """Enable 3 * v (reverse multiplication)."""
        return self * scalar
    
    def __repr__(self) -> str:
        return f"Vector({self.x}, {self.y})"

v1: Vector = Vector(1, 2)
v2: Vector = Vector(3, 4)
print(v1 + v2)      # Vector(4, 6)
print(v1 * 3)       # Vector(3, 6)
print(3 * v1)       # Vector(3, 6) - uses __rmul__
```

## 6.8 Abstract Base Classes (ABCs)

ABCs define interfaces—classes that specify methods derived classes must implement without providing implementation themselves. This enforces a contract for subclasses.

```python
from abc import ABC, abstractmethod
from typing import final

class Shape(ABC):
    """
    Abstract base class - cannot be instantiated directly.
    Subclasses must implement all abstract methods.
    """
    
    @abstractmethod
    def area(self) -> float:
        """Calculate area. Must be implemented by subclasses."""
        pass
    
    @abstractmethod
    def perimeter(self) -> float:
        """Calculate perimeter. Must be implemented by subclasses."""
        pass
    
    @final
    def description(self) -> str:
        """
        Concrete method in ABC.
        @final prevents subclasses from overriding (Python 3.8+).
        """
        return f"Shape with area {self.area()}"

class Rectangle(Shape):
    def __init__(self, width: float, height: float) -> None:
        self.width: float = width
        self.height: float = height
    
    def area(self) -> float:
        return self.width * self.height
    
    def perimeter(self) -> float:
        return 2 * (self.width + self.height)

# Usage
rect: Rectangle = Rectangle(5, 3)
print(rect.area())          # 15
print(rect.description())   # "Shape with area 15.0"

# shape: Shape = Shape()    # TypeError: Can't instantiate abstract class
```

**Protocols and Structural Subtyping (Python 3.8+):**
For a more flexible "duck typing" approach without strict inheritance, use `typing.Protocol` (Structural Pattern Matching):

```python
from typing import Protocol

class Drawable(Protocol):
    """Anything that has a draw method satisfies this protocol."""
    def draw(self) -> None: ...

def render(item: Drawable) -> None:
    """Accepts any object with a draw method, regardless of inheritance."""
    item.draw()

class Circle:
    def draw(self) -> None:
        print("Drawing circle")

# Circle doesn't inherit from Drawable, but satisfies the protocol
render(Circle())  # Valid
```

## Summary

Object-oriented programming in Python provides powerful tools for organizing complex systems. You have learned to define **classes** with proper initialization, distinguish between **instance, class, and static methods**, and leverage **inheritance** (preferring composition when appropriate). You understand Python's **MRO** for navigating multiple inheritance and use **name mangling** and conventions for encapsulation.

The **@property** decorator enables you to expose controlled interfaces to your data, while **magic methods** integrate your objects seamlessly with Python's syntax and built-in functions. Finally, **Abstract Base Classes** allow you to define interfaces and enforce contracts, ensuring consistent APIs across implementations.

However, writing code is only half the battle. Professional software requires consistency, clarity, and collaboration. In the next chapter, we will examine the standards and tools that elevate Python code from functional to professional: PEP 8 style guidelines, type hinting strategies, documentation standards, and code quality automation.

**Next Chapter**: Chapter 7: Code Style, Type Hinting, and Quality Assurance.